<a href="https://colab.research.google.com/github/alejovillores/tp2_spark_villores_alejo/blob/main/ejercicio_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 30
Qué porcentaje de contenido geolocalizado de wikipedia NO está en la tierra

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 60.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=54fd57df25c330f83da8305a127314d5365f45a7c25d59a81e230a643dc95efb
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release 

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import  *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [5]:
sqlContext = SQLContext(sc)
df = sqlContext.read.parquet('/content/drive/MyDrive/2C2021/Datos/Trabajo Practico 2/geo_tags.csv', header=True, inferSchema=True)
rdd_geo_tags = df.rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [6]:
rdd_geo_tags_contents_cleaned = rdd_geo_tags.filter(lambda x: x.gt_page_id != None).filter(lambda x: x.gt_globe != None)

In [7]:
rdd_geo_tags_contents_cleaned.count()

556393

In [ ]:
help(rdd_geo_tags)

Busco el porcentaje de los contenidos distintos de la tierra

In [8]:
mean_out_of_earth = rdd_geo_tags_contents_cleaned.map(lambda x: ('no-earth' if x.gt_globe != 'earth' else 'earth',1))\
.reduceByKey(lambda x,y: x+y)

In [9]:
mean_out_of_earth.take(2)

[('earth', 545154), ('no-earth', 11239)]

In [10]:
res = mean_out_of_earth.filter(lambda x : x[0] == 'no-earth').map(lambda x: x[1]).reduce(lambda x: x[1]) / rdd_geo_tags_contents_cleaned.count()
print('El porcentaje de los contenidos fuera de la tierra será de: % ', str(res*100))

El porcentaje de los contenidos fuera de la tierra será de: %  2.0199750895500124
